In [1]:
import numpy as np

In [ ]:
class Node:
         
    def __init__(self, initial, parent=None, acao=None, custo=0):

        self.initial = initial
    
    def __repr__(self):
        return "<Node {}>".format(self.initial)

    def expansao(self, problema, rainha):
        
        return [self.child_node(problema, rainha, acao) 
                for acao in problema.pos_mov()]
        

    def child_node(self, problem, rainha, acao):

        prox_estado = problem.jogada(self.initial, rainha, acao)

        prox_node = Node(prox_estado, self, acao, 0)

        return prox_node